In [18]:
global_parameters = {
    # 날짜별 에러 정보를 유저 아이디로 취합할 때 사용할 quantile 값
    'quantile': [0.10, 0.25, 0.35, 0.75, 0.80, 0.90],
    
    # LGBM 앙상블 학습시 사용할 fold 개수
    'nfold': 10,
    
    # STMOE 알고리즘에 대해 멀티 프로세싱을 사용할지 여부, (workers.py 파일이 존재해야함)
    'multiprocessing_for_smote': True,
    
    # 데이터 세트에 x개 이상 정보가 에러타입_코드 칼럼만 사용
    'min_errcode_sample': 75,
    
    # pearson 상관계수를 이용하여 x 값 이상인 칼럼을 제거함 (1을 초과할 경우 작동하지 않음)
    'pearson_cutoff': 1,
    
    # lgbm에서 seed를 변경하여 앙상블 모델의 다양성 추가 (기본값1)
    'lgbm_seed_ensemble': 1,
}

In [2]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import collections

warnings.filterwarnings(action='ignore')

PATH = '../data/'
def make_days(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    return (dt.date(year, month, day) - dt.date(2020, 10, 31)).days

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^\-0-9\.]+", '', str(x))
    if x =='':
        return -1
    else:
        return int(float(x))

In [3]:
train_err = pd.read_csv(PATH+'train_err_data.csv')
train_err['days'] = train_err['time'].apply(make_days)
test_err = pd.read_csv(PATH+'test_err_data.csv')
test_err['days'] = test_err['time'].apply(make_days)

In [4]:
# 문자열(ex "1,000")로 되어있는 퀄리티 데이터 float 타입으로 변환
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
for i in range(0,13,1):
    train_quality['quality_' + str(i)] = train_quality['quality_' + str(i)].apply(lambda x: string2num(x)).astype(float)
train_quality['days'] = train_quality['time'].apply(make_days)

test_quality = pd.read_csv(PATH+'test_quality_data.csv')
for i in range(0,13,1):
    test_quality['quality_' + str(i)] = test_quality['quality_' + str(i)].apply(lambda x: string2num(x)).astype(float)
test_quality['days'] = test_quality['time'].apply(make_days)

In [5]:
# [user_id, group]
quality_user_id = {}
for i in range(10000,45000):
    quality_user_id[i] = []
    for j in range(0,13):
        quality_user_id[i].append([])
        
for i, group in train_quality[train_quality.columns.difference(['time'])].groupby(['user_id']):
    for j in range(0,13):
        quality_user_id[i][j] = group.loc[group['quality_'+str(j)] != -1]['quality_'+str(j)].values
        
for i, group in test_quality[test_quality.columns.difference(['time'])].groupby(['user_id']):
    for j in range(0,13):
        quality_user_id[i][j] = group.loc[group['quality_'+str(j)] != -1]['quality_'+str(j)].values
        
quality_columns_preset =  ['quality_count'] + ['quality_sum'] + ['quality_mean']  + ['quality_var'] + ['quality_max'] + ['quality_std/mean']
quality_user_id_statistics = {}
for i in quality_user_id:
    quality_user_id_statistics[i] = []
    for j in range(0,13):
        a = np.array(quality_user_id[i][j])
        if (len(a) == 0):
            quality_user_id_statistics[i] += [0] + [-999] * (len(quality_columns_preset) - 1)
        else: 
            quality_user_id_statistics[i] += [len(a), a.sum(axis=0), a.mean(axis=0), a.var(axis=0), a.max(axis=0), a.std(axis=0) / a.mean(axis=0)]
quality_columns = []
quality_columns_index = {}
for i in range(0,13):
    for j in quality_columns_preset:
        # 어차피 이미 만들어진 퀄리티 데이터에 접근하지 않으니 index가 필요없음
        # quality_columns_index[column_name] = len(quality_columns)
        quality_columns.append(j + '_' + str(i))

In [6]:
# 33일 넘기는 테스트 데이터 제거
test_err = test_err.loc[test_err['days'] < 33]

In [7]:
# white_list = {(25, 'scanning timeout'), (8, 'PHONE_ERR'), (36, '8.0'), (5, 'Q-64002'), (19, '1'), (30, '1'), (25, 'connectionterminated by local host'), (21, '1'), (1, 'P-41007'), (25, '2'), (5, 'Q-64001'), (4, '0'), (37, '0'), (25, '1'), (25, 'UNKNOWN'), (18, '1'), (17, '21'), (1, '0'), (20, '1'), (1, 'P-44010'), (5, 'U-82026'), (3, '0'), (23, 'terminate by peer user')}
white_list_code = {'Q-64001', 'S-65002', '21', 'Q-64002', 'PHONE_ERR', '3', 'scanning timeout', 'NFANDROID2', '2', 'U-82026', 'P-41007', 'V-21008', 'P-44010', '8.0', 'terminate by peer user'}

In [8]:
test_type_code = set()
for i, group in test_err.groupby(['errtype', 'errcode']):
    if len(group['user_id'].unique()) >= global_parameters['min_errcode_sample']:
        test_type_code.add((i[0], i[1]))

same_type_code = []
for i, group in train_err.groupby(['errtype', 'errcode']):
    if len(group['user_id'].unique()) >= global_parameters['min_errcode_sample'] and (i[0], i[1]) in test_type_code:
        same_type_code.append((i[0],i[1]))

In [9]:
# error_type, error_code 매핑
def key_typecode(t,c = None):
    if c == None:
        return "errcode_" + str(t).strip()
    
    return str(t).strip() + '_' + str(c).strip()

columns = []
column_index = {}
count = 0
for i in same_type_code:
    key = key_typecode(i[0], i[1])
    if key not in columns:
        columns.append(key)
        column_index[key] = count
        count += 1
'''        
for i in white_list_code:
    key = key_typecode(i)
    if key not in columns:
        columns.append(key)
        column_index[key] = count
        count += 1
        '''
            
print("검색된 칼럼 수", len(columns))

검색된 칼럼 수 99


In [22]:
def make_err_table(table, start_user_id, size):    
    id_error = table[['user_id','errtype','errcode', 'days']].values
    error = np.zeros((size, 33, 42 + len(columns)))    
    
    days = np.zeros((size, 33))
    
    for person_idx, err, code, day in tqdm(id_error):
        user_index = person_idx - start_user_id
        # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
        error[user_index, day, err - 1] += 1
        days[user_index, day] = 1
        key = key_typecode(err - 1, code)
        if key in column_index:
            error[user_index, day, 42 + column_index[key]] += 1
           
        # 에러 코드별 합계를 추가하고싶으면 아래 주석 해제
        '''
        key = key_typecode(code)
        if key in column_index:
            error[person_idx - start_user_id, day_key, 42 + column_index[key]] += 1
        '''
       
    
    error_sum = np.sum(error, axis=1)
    error_std = np.std(error, axis=1)
    error_max = np.max(error, axis=1)
    
    error_cv = np.std(error, axis=1) / np.mean(error, axis=1)
    
    error_cv[np.isnan(error_cv)] = 0
    
    quantile_arr = np.quantile(error, global_parameters['quantile'], axis=1)
    #sum_days = np.sum(days, axis=1).reshape(-1,1)
    
    # weekday_error_sum = np.sum(weekday_error, axis=1)
    # weekday_error_std = np.std(weekday_error, axis=1)
    # weekday_error_max = np.max(weekday_error, axis=1)
    
     
    total = np.concatenate([error_sum, error_std, error_max, error_cv] + [i for i in quantile_arr]
                           , axis=1)
    quantile_columns = []
    temp_columns = [str(i) for i in range(0,42)] + columns
    for q in global_parameters['quantile']:
        quantile_columns.extend([str(int(q * 100)) + '_' + str(i) for i in temp_columns])
        
    
    result = pd.DataFrame(data=total, columns=
                          ['sum_' + str(i) for i in temp_columns] + 
                          ['std_' + str(i) for i in temp_columns] + 
                          ['max_' + str(i) for i in temp_columns] +
                          ['cv_' + str(i) for i in temp_columns] +
                          quantile_columns)
    
    # 에러와 관련이 없는 부가 정보 입력
    
    # quality 데이터 연결
    result['user_id'] = range(start_user_id, start_user_id + size)
    for i in tqdm(range(0,len(quality_columns))):
        result.loc[:, quality_columns[i]] = result['user_id'].apply(lambda x: quality_user_id_statistics[x][i])
    
    del result['user_id'] # user_id는 학습에 사용되지 않아야하기 때문에 삭제
    return result

train_full = make_err_table(train_err, start_user_id=10000, size=15000)


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 143.09it/s]


In [23]:
test = make_err_table(test_err, start_user_id=30000, size=14999)

100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 172.15it/s]


In [24]:
train_full

,sum_0,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,sum_7,sum_8,sum_9,sum_10,sum_11,sum_12,sum_13,sum_14,sum_15,sum_16,sum_17,sum_18,sum_19,sum_20,sum_21,sum_22,sum_23,sum_24,sum_25,sum_26,sum_27,sum_28,sum_29,sum_30,sum_31,sum_32,sum_33,sum_34,sum_35,sum_36,sum_37,sum_38,sum_39,sum_40,sum_41,sum_10_0,sum_10_10,sum_10_11,sum_10_14,sum_10_15,sum_10_21,sum_10_22,sum_10_25,sum_10_3,sum_10_30,sum_10_31,sum_10_32,sum_10_39,sum_10_39_0,sum_10_39_1,sum_25_0,sum_25_10,sum_25_11,sum_25_14,sum_25_15,sum_25_21,sum_25_22,sum_25_25,sum_25_26,sum_25_27,sum_25_3,sum_25_30,sum_25_31,sum_25_32,sum_25_39,sum_25_39_0,sum_25_39_1,sum_25_3_1,sum_25_4,sum_25_40,sum_35_0,sum_35_10,sum_35_11,sum_35_13,sum_35_14,sum_35_15,sum_35_21,sum_35_22,sum_35_25,sum_35_26,sum_35_27,sum_35_3,sum_35_30,sum_35_31,sum_35_32,sum_35_33,sum_35_39,sum_35_39_0,sum_35_39_1,sum_35_3_1,sum_35_4,sum_35_40,sum_75_0,sum_75_10,sum_75_11,sum_75_13,sum_75_13_1,sum_75_14,sum_75_15,sum_75_16,sum_75_16_1,sum_75_2,sum_75_21,sum_75_22,sum_75_23,sum_75_25,sum_75_26,sum_75_27,sum_75_2_1,sum_75_3,sum_75_30,sum_75_31,sum_75_32,sum_75_33,sum_75_33_1,sum_75_38,sum_75_39,sum_75_39_0,sum_75_39_1,sum_75_3_1,sum_75_4,sum_75_40,sum_75_41,sum_75_5,sum_75_6,sum_75_6_1,sum_75_6_14,sum_75_9,sum_80_0,sum_80_10,sum_80_11,sum_80_12,sum_80_13,sum_80_13_1,sum_80_14,sum_80_15,sum_80_16,sum_80_16_1,sum_80_2,sum_80_21,sum_80_22,sum_80_23,sum_80_25,sum_80_26,sum_80_27,sum_80_2_1,sum_80_3,sum_80_30,sum_80_31,sum_80_32,sum_80_33,sum_80_33_1,sum_80_38,sum_80_39,sum_80_39_0,sum_80_39_1,sum_80_3_1,sum_80_4,sum_80_40,sum_80_41,sum_80_5,sum_80_6,sum_80_6_1,sum_80_6_14,sum_80_9,sum_90_0,sum_90_10,sum_90_11,sum_90_12,sum_90_13,sum_90_13_1,sum_90_14,sum_90_15,sum_90_16,sum_90_16_1,sum_90_2,sum_90_21,sum_90_22,sum_90_23,sum_90_25,sum_90_26,sum_90_27,sum_90_2_1,sum_90_3,sum_90_30,sum_90_31,sum_90_32,sum_90_33,sum_90_33_1,sum_90_33_2,sum_90_33_3,sum_90_34,sum_90_37,sum_90_38,sum_90_39,sum_90_39_0,sum_90_39_1,sum_90_3_1,sum_90_4,sum_90_40,sum_90_41,sum_90_5,sum_90_6,sum_90_6_1,sum_90_6_14,sum_90_9,sum_cv_0,sum_cv_1,sum_cv_10,sum_cv_11,sum_cv_12,sum_cv_13,sum_cv_13_1,sum_cv_14,sum_cv_15,sum_cv_16,sum_cv_16_1,sum_cv_17,sum_cv_18,sum_cv_19,sum_cv_1_0,sum_cv_2,sum_cv_20,sum_cv_21,sum_cv_22,sum_cv_23,sum_cv_24,sum_cv_24_1,sum_cv_25,sum_cv_26,sum_cv_27,sum_cv_29,sum_cv_2_1,sum_cv_3,sum_cv_30,sum_cv_31,sum_cv_32,sum_cv_33,sum_cv_33_1,sum_cv_33_2,sum_cv_33_3,sum_cv_34,sum_cv_35,...,90_max_41,90_max_5,90_max_6,90_max_6_1,90_max_6_14,90_max_9,90_quality_count_0,90_quality_count_1,90_quality_count_10,90_quality_count_11,90_quality_count_2,90_quality_count_5,90_quality_count_6,90_quality_max_0,90_quality_max_1,90_quality_max_10,90_quality_max_11,90_quality_max_12,90_quality_max_2,90_quality_max_5,90_quality_max_6,90_quality_max_7,90_quality_max_8,90_quality_max_9,90_quality_mean_0,90_quality_mean_1,90_quality_mean_10,90_quality_mean_11,90_quality_mean_12,90_quality_mean_2,90_quality_mean_5,90_quality_mean_6,90_quality_mean_7,90_quality_mean_8,90_quality_mean_9,90_quality_std/mean_0,90_quality_std/mean_1,90_quality_std/mean_10,90_quality_std/mean_11,90_quality_std/mean_12,90_quality_std/mean_2,90_quality_std/mean_5,90_quality_std/mean_6,90_quality_std/mean_7,90_quality_std/mean_8,90_quality_std/mean_9,90_quality_sum_0,90_quality_sum_1,90_quality_sum_10,90_quality_sum_11,90_quality_sum_12,90_quality_sum_2,90_quality_sum_5,90_quality_sum_6,90_quality_sum_7,90_quality_sum_8,90_quality_sum_9,90_quality_var_0,90_quality_var_1,90_quality_var_10,90_quality_var_11,90_quality_var_12,90_quality_var_2,90_quality_var_5,90_quality_var_6,90_quality_var_7,90_quality_var_8,90_quality_var_9,90_std_0,90_std_1,90_std_10,90_std_11,90_std_12,90_std_13,90_std_13_1,90_std_14,90_std_15,90_std_16,90_std_16_1,90_std_17,90_std_18,90_std_19,90_std_1_0,90_std_2,90_std_20,90_std_21,90_std_22,90_std_23,90_std_24,90_std_24_1,90_std_25,90_std_26,90_std_27,90_std_29,90_std_2_1,90_std_3,90_std_30,90_std_31,90_std_32,90_std_33,90_std_33_1,90_std_33_2,90_std_33_3,90_std_34,90_std

In [25]:
for i in range(13):
    if "quality_std/mean_"+str(i) in train_full.columns:
        train_full["quality_std/mean_"+str(i)] = train_full["quality_std/mean_"+str(i)].fillna(0)

In [26]:
for i in range(13):
    if "quality_std/mean_"+str(i) in test.columns:
        test["quality_std/mean_"+str(i)] = test["quality_std/mean_"+str(i)].fillna(0)


In [27]:
# 사용되지 않는 타입_코드 칼럼을 제거
#train_test = pd.concat([train_full, test], axis=0)
select_columns = []
for c in tqdm(train_full.columns):
    if str(train_full[c].dtype) == 'category':
        select_columns.append(c)
        continue
    if train_full[c].apply(lambda x: 1 if x > 0 else 0).sum() >= global_parameters['min_errcode_sample'] and test[c].apply(lambda x: 1 if x > 0 else 0).sum() >= global_parameters['min_errcode_sample']: # 트레이닝, 테스트 세트에 50개 이상의 행이 있어야함
        select_columns.append(c)
    else:
        if 'custom_' in c:
            select_columns.append(c)
        '''
        # 코드 합계는 모두 통과 (화이트리스트)
        if 'sum_' in c or 'max_' in c or 'std_' in c:
            if 'errcode_' in c:
                select_columns.append(c)
                
        '''
        '''
        if 'sum_' in c or 'max_' in c or 'std_' in c:
            original_columns = c[4:].split('_')
            if len(original_columns) == 2 and original_columns[0] != 'errcode' and (int(original_columns[0]), original_columns[1]) in white_list:
                if 'sum_' in c:
                    print(c)
                select_columns.append(c)
        '''

print(len(train_full.columns), '개의 칼럼을', len(select_columns), '개로 압축')
train_full = train_full[select_columns]
test = test[select_columns]
columns = select_columns

100%|█████████████████████████████████████████████████████████████████████████████| 4908/4908 [00:25<00:00, 192.75it/s]


4908 개의 칼럼을 552 개로 압축


In [28]:
if global_parameters['pearson_cutoff'] <= 1:
    org_train_full = train_full
    pearson_table = train_full.corr(method='pearson')
    pearson = []
    for i in train_full.columns:
        for j in train_full.columns:
            if (i >= j):
                continue
            if str(train_full[i].dtype) == 'category' or str(train_full[j].dtype) == 'category':
                continue
            pearson.append((i,j,pearson_table.loc[i,j]))
    pearson = sorted(pearson, key=lambda x: abs(x[2]), reverse=True)

    remove_columns = set()
    fix_columns = set()
    for i in pearson:
        # 이미 제거되었으면 넘기기
        if (i[0] in remove_columns or i[1] in remove_columns):
            continue
        if (abs(i[2]) >= global_parameters['pearson_cutoff']):
            remove_columns.add(i[1])

    print(len(train_full.columns), '개의 칼럼을', len(train_full.columns) - len(remove_columns), '개로 압축')
    train_full = train_full[train_full.columns.difference(list(remove_columns))]
    test = test[test.columns.difference(list(remove_columns))]


552 개의 칼럼을 441 개로 압축


In [36]:
train_full

,10_0,10_10,10_11,10_14,10_15,10_21,10_22,10_25,10_3,10_30,10_31,10_32,10_39,10_75_9,10_80_0,25_0,25_10,25_10_21,25_11,25_14,25_15,25_21,25_22,25_25,25_26,25_27,25_3,25_30,25_31,25_32,25_39,25_4,25_40,25_75_9,25_80_0,35_0,35_10,35_10_21,35_11,35_13,35_14,35_15,35_21,35_22,35_25,35_26,35_27,35_3,35_30,35_31,35_32,35_33,35_39,35_4,35_40,35_75_9,35_80_0,75_0,75_10,75_10_15,75_10_21,75_11,75_13,75_14,75_15,75_16,75_2,75_21,75_22,75_23,75_25,75_25_15,75_25_21,75_25_30,75_25_39_1,75_26,75_27,75_3,75_30,75_31,75_32,75_33,75_38,75_39,75_4,75_40,75_41,75_5,75_6,75_75_38,75_75_9,75_80_0,75_9,80_0,80_10,80_10_15,80_10_21,80_11,80_12,80_13,80_14,80_15,80_16,80_2,80_21,80_22,80_23,80_25,80_25_15,80_25_21,80_25_30,80_25_39_1,80_26,80_27,80_3,80_30,80_31,80_32,80_33,80_38,80_39,80_4,80_40,80_41,80_5,80_6,80_75_38,80_75_9,80_80_0,80_9,90_0,90_10,90_10_15,90_10_21,90_11,90_12,90_13,90_14,90_15,90_16,90_2,90_21,90_22,90_23,90_25,90_25_15,90_25_21,90_25_30,90_25_39_1,90_26,90_27,90_3,90_30,90_31,90_32,90_33,90_34,90_37,90_38,90_39,90_4,90_40,90_41,90_5,90_6,90_75_38,90_75_39,90_75_39_0,90_75_9,90_80_0,90_9,cv_0,cv_1,cv_10,cv_10_0,cv_10_15,cv_10_21,cv_11,cv_12,cv_13,cv_14,cv_15,cv_16,cv_17,cv_18,cv_19,cv_2,cv_20,cv_21,cv_22,cv_23,cv_24,cv_25,cv_25_15,cv_25_21,cv_25_30,cv_25_39_1,cv_26,cv_27,cv_29,cv_3,cv_30,cv_31,cv_32,cv_33,cv_34,cv_35,cv_35_32,cv_36,cv_37,cv_38,cv_39,cv_4,cv_40,cv_41,cv_5,cv_6,cv_75_38,cv_75_39,cv_75_39_0,cv_75_9,cv_80_0,cv_9,max_0,max_1,max_10,max_10_0,max_10_15,max_10_21,max_11,max_12,max_13,max_14,max_15,max_16,max_17,max_18,max_19,max_2,max_20,max_21,max_22,max_23,max_24,max_25,max_25_15,max_25_21,max_25_30,max_25_39_1,max_26,max_27,max_29,max_3,max_30,max_31,max_32,max_33,max_34,max_35,max_35_32,max_36,max_37,max_38,max_39,max_4,max_40,max_41,max_5,max_6,max_75_38,max_75_39,max_75_39_0,max_75_9,max_80_0,max_9,quality_count_0,quality_count_1,quality_count_10,quality_count_11,quality_count_2,quality_count_5,quality_count_6,quality_max_0,quality_max_1,quality_max_10,quality_max_11,quality_max_12,quality_max_2,quality_max_5,quality_max_6,quality_max_7,quality_max_8,quality_max_9,quality_mean_0,quality_mean_1,quality_mean_10,quality_mean_11,quality_mean_12,quality_mean_2,quality_mean_5,quality_mean_6,quality_mean_7,quality_mean_8,quality_mean_9,quality_std/mean_0,quality_std/mean_1,quality_std/mean_10,quality_std/mean_11,quality_std/mean_12,quality_std/mean_2,quality_std/mean_5,quality_std/mean_6,quality_std/mean_7,quality_std/mean_8,quality_std/mean_9,quality_sum_0,quality_sum_1,quality_sum_10,quality_sum_11,quality_sum_12,quality_sum_2,quality_sum_5,quality_sum_6,quality_sum_7,quality_sum_8,quality_sum_9,quality_var_0,quality_var_1,quality_var_10,quality_var_11,quality_var_12,quality_var_2,quality_var_5,quality_var_6,quality_var_7,quality_var_8,quality_var_9,std_0,std_1,std_10,std_10_0,std_10_15,std_10_21,std_11,std_12,std_13,std_14,std_15,std_16,std_17,std_18,std_19,std_2,std_20,std_21,std_22,std_23,std_24,std_25,std_25_15,std_25_21,std_25_30,std_25_39_1,std_26,std_27,std_29,std_3,std_30,std_31,std_32,std_33,std_34,std_35,std_35_32,std_36,std_37,std_38,std_39,std_4,std_40,std_41,std_5,std_6,std_75_38,std_75_39,std_75_39_0,std_75_9,std_80_0,std_9,sum_0,sum_1,sum_10,sum_10_0,sum_10_15,sum_10_21,sum_11,sum_12,sum_13,sum_14,sum_15,sum_16,sum_17,sum_18,sum_19,sum_2,sum_20,sum_21,sum_22,sum_23,sum_24,sum_25,sum_25_15,sum_25_21,sum_25_30,sum_25_39_1,sum_26,sum_27,sum_29,sum_3,sum_30,sum_31,sum_32,sum_33,sum_34,sum_35,sum_35_32,sum_36,sum_37,sum_38,sum_39,sum_4,sum_40,sum_41,sum_5,sum_6,sum_75_38,sum_75_39,sum_75_39_0,sum_75_9,sum_80_0,sum_9
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.2,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,

In [112]:
def make_correlation(data, threshold, target_column, methods):
    re = data.corr(method = methods)
    is_po = re[re.columns & target_column] > threshold
    is_ne = re[re.columns & target_column] < -threshold
    
    print("Method", methods)
    result_co_p = re[is_po][re.columns & target_column].drop(is_po.index & target_column)
    result_co_n = re[is_ne][re.columns & target_column].drop(is_ne.index & target_column)
    
    
    for i in result_co_p:
        for j in range(len(result_co_p[i])):
            if result_co_p[i][j] >threshold:
                print("Correlation : ", i, "&", result_co_p[i].index[j], result_co_p[i][j])
    for i in result_co_n:
        for j in range(len(result_co_n[i])):
            if result_co_n[i][j] < - threshold:
                print("Correlation : ", i, "&", result_co_n[i].index[j], result_co_n[i][j])
            

In [115]:
# 0.3 이상이 강한 상관관계
make_correlation(train_full, 0.23, quality_columns, "pearson")

Method pearson
Correlation :  quality_count_2 & 25_10_21 0.23229000939597824
Correlation :  quality_count_2 & 25_26 0.23180925564123558
Correlation :  quality_count_2 & 25_27 0.23123257010987497
Correlation :  quality_max_6 & cv_26 0.24691724388632777
Correlation :  quality_max_6 & cv_27 0.24729121455055306
Correlation :  quality_max_6 & cv_34 0.24289588425900843
Correlation :  quality_max_7 & cv_26 0.23857546539922006
Correlation :  quality_max_7 & cv_27 0.2386392443566284
Correlation :  quality_max_7 & cv_34 0.2302298259598735
Correlation :  quality_mean_7 & cv_26 0.23113187172103783
Correlation :  quality_mean_7 & cv_27 0.2313244051029107
Correlation :  quality_std/mean_0 & cv_34 0.24194542240512715
Correlation :  quality_std/mean_2 & cv_27 0.23025086545456566


In [118]:
# 단순히 한 변수가 증가 할 때 다른 변수가 증가하는지 감소하는지에 대한 관계 
# 예) 성적 등수로 매겼을 때,, 언어 영역과 수리 영역이 관계가 있는지

make_correlation(train_full, 0.3, quality_columns, "spearman")

Method spearman
Correlation :  quality_count_1 & cv_26 0.3149437660303116
Correlation :  quality_count_1 & cv_27 0.3154221959808971
Correlation :  quality_count_1 & max_26 0.3223422267291956
Correlation :  quality_count_1 & max_27 0.32282317988516923
Correlation :  quality_count_1 & std_26 0.32363944074519724
Correlation :  quality_count_1 & std_27 0.3241173295893633
Correlation :  quality_count_1 & sum_26 0.32386445512029294
Correlation :  quality_count_1 & sum_27 0.3243398012720819
Correlation :  quality_count_10 & cv_26 0.3168370229637717
Correlation :  quality_count_10 & cv_27 0.3172569213650514
Correlation :  quality_count_10 & max_26 0.3240799369019483
Correlation :  quality_count_10 & max_27 0.3245158284902758
Correlation :  quality_count_10 & std_26 0.3254252352075094
Correlation :  quality_count_10 & std_27 0.3258549809446329
Correlation :  quality_count_10 & sum_26 0.32560982624086565
Correlation :  quality_count_10 & sum_27 0.3260382935033147
Correlation :  quality_count_11 